In [2]:
import polars as pl
import dtale
import polars.selectors as cs
import plotly.express as px
import sweetviz
import sklearn
import xgboost

c:\Users\farshid.afshar\AppData\Local\anaconda3\envs\pl_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [3]:
df_raw=pl.read_csv(r'C:\Users\farshid.afshar\Jet\Personal\ForceAI/insurance_data_sample.csv')\
    .with_columns(Date=pl.col('Date').str.to_datetime("%m/%d/%Y").dt.date(),
                  Female_dummy=pl.when(pl.col('Gender')=='Female').then(1).otherwise(0))

### handle numerci varbs

In [17]:
## according to eda report phone number is redundent
''' 
the above mentioned variable according to eda are less effective in claim decision
'''
df_numeric=df_raw.select(cs.integer(),cs.float()).drop('Phone')\
                .with_columns(cliam_indicator=pl.when(pl.col('Claim amount')>0).then(1).otherwise(0))

### handle non numeric vaerbs

In [20]:
## for those who have many cats just select top two with most claims
## lets discard Car_id and customer name, gender, dealer name, Dealer_No


non_numeric_elim=['Car_id','Customer Name','Gender','Dealer_Name','Model','Engine','Dealer_No']

''' 
the above mentioned variable according to eda are less effective in claim decision
'''
df_non_numeric=df_raw.with_columns(cliam_indicator=pl.when(pl.col('Claim amount')>0).then(1).otherwise(0))\
                .select([t for t in df_raw.columns if t not in df_numeric.columns+['Date']]).drop(non_numeric_elim)
                

In [21]:
df_non_numeric

Company,Transmission,Color,Dealer_No,Body Style,Phone,City
str,str,str,str,str,i64,str
"""Ford""","""Auto""","""Black""","""06457-3834""","""SUV""",8264678,"""Riga"""
"""Dodge""","""Auto""","""Black""","""60504-7114""","""SUV""",6848189,"""Liepaja"""
"""Cadillac""","""Manual""","""Red""","""38701-8047""","""Passenger""",7298798,"""Riga"""
"""Toyota""","""Manual""","""Pale White""","""99301-3882""","""SUV""",6257557,"""Jelgava"""
"""Acura""","""Auto""","""Red""","""53546-9427""","""Hatchback""",7081483,"""Liepaja"""
"""Mitsubishi""","""Manual""","""Pale White""","""85257-3102""","""Hatchback""",7315216,"""Riga"""
"""Toyota""","""Manual""","""Pale White""","""78758-7841""","""Passenger""",7727879,"""Riga"""
"""Mitsubishi""","""Auto""","""Pale White""","""78758-7841""","""Passenger""",6206512,"""Jelgava"""
"""Chevrolet""","""Manual""","""Pale White""","""85257-3102""","""Hardtop""",7194857,"""Ventspils"""
